# Python Scientific Data Analysis
## Course's Final Project
### Barak Daniel - 204594329

## Installations needed for the program to run:

In [ ]:
#!pip install numpy
#!pip install pandas
#!pip install seaborn
#!pip install matplotlib
#!pip install seaborn
#!pip install sklearn
#!pip install scipy
#!pip install pydotplus

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.naive_bayes import GaussianNB
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pydotplus

# Intro

### Overview
This project is about targetted marketing, in the data set given we have 'customer' as each row of data, and the features can tell us about the customer's status in general (age, marriage, etc..) and hes shopping behavior.
In this section of the project I will go over the data set and the goal to try and understand the whole process before starting the actual work on the data.

### So what exactly is targetted marketing?
Targeting in marketing is a strategy that breaks a large market into smaller segments to concentrate on a specific group of customers within that audience. It defines a segment of customers based on their unique characteristics and focuses solely on serving them.
Instead of trying to reach an entire market, a brand uses target marketing to put their energy into connecting with a specific, defined group within that market.
So for this reason I'll break down all the features in the given dataset ('customers3.csv'), and understand them.


### Feature's data breadown
The following features are included in the data set given in 'customes3.csv':
- ID - Unique ID to each customer
- Gender - The gender of the customer
- Ever_Marries - Indicates if the customer was married
- Age - The age of the customer
- Graduated - Has the customer graduated high school
- Profession - The profession of the customer
- Work_Experience - The number of years of the customer's expirence in his profession
- Spending_Score - The spending habits of the customer classified to 3 categories
- Family_Size - The number of family members the customer has in his household
- Shop_Day - The day of the week which the customer is shopping on the most
- Shop_Other - Normalized measure of customer deviation from average store customer spending on non specified products
- Shop_Dairy - Normalized measure of customer deviation from average store customer spending on dairy products
- Shop_Household: Normalized measure of customer deviation from average store customer spending on household products
- Shop_Meat - Normalized measure of customer deviation from average store customer spending on meat products
- Group - The target group which the customer belongs to

### Feature's type breakdown
- ID - Numerical discrete (Integer)
- Gender - Categorical (Male/Female)
- Ever_Marries - Categorical nominal (Yes/No)
- Age - Numerical continuous (Integer)
- Graduated - Categorical (Yes/No)
- Profession - Categorical nominal
- Work_Experience - Numerical discrete (Integer)
- Spending_Score - Categorical ordinal (Low/average/High)
- Family_Size - Numerical discrete (Integer)
- Shop_Day - Categorical ordinal (Sunday, Monday, ..., Saturday)
- Shop_Other - Numerical continuous (Double)
- Shop_Dair - Numerical continuous (Double)
- Shop_Household - Numerical continuous (Double)
- Shop_Meat -Numerical continuous (Double)
- Group - Categorical nominal

In [ ]:
df = pd.read_csv('customers3.csv')
count = df.count()

print("The number of rows is: {}".format(len(df.index)))
print("The number of columns is: {}".format(len(df.columns)))
print("The number of cells is: {}".format(len(df.index) * len(df.columns)))
print("The number of cells with concrete values is: {}".format(count.sum()))
print("The number of cells without concrete values is: {}\n".format(len(df.index) * len(df.columns) - count.sum()))

print("\nThe number of concrete values for each feature:")
df.count()


### The size of the data set is:
- 8120 rows of customer's data (+1 for the headers of each column)
- 15 columns for the features
- 8120*15 = 121,8000 cells, but we can see that not all of them has concrete values.

### Missing values:
After watching the dataset and trying to understand it, I have also encountered many cells with missing data values.
After reading the dataset a transformation of this Nan values is needed, for each feature with missing data, I'll examine it and understand which of the methods is the best to deal with those values (Mode, Mean, Median, Removal, etc..).

### Other types of missing values:
A validation for the values that are not missing must be made, after going through the features, the options are numeric value which is out of the range as given with feature definition, a numeric value that cannot be negative, etc...

After going through out the dataset, those are the features needed to be fixed:
- Shop_Day - Must contain values of 1 to 7 but there are values out of this range therefore it will be filled by the same method as all the feature values
- Shop_Household - There are values of '-0' and 0 as well, so a replacement with 0 value will be good.
- Shop_Diary - There are values of '-0' and 0 as well, so a replacement with 0 value will be good.
- Shop_Other - There are values of '-0' and 0 as well, so a replacement with 0 value will be good.


## Initial Data Analysis

As we saw above, there are a lot of missing values and categorical values we want to transform before we can alanyze the data completely.
In this section of the project I will deal with those values, for each feature a check for the accuracy of the model will be taken and by that I can make the decision what was the best method for the feature.